In [57]:

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers   import TensorBoardLogger


from omegaconf import DictConfig
from ruamel.yaml import YAML

import os
import torch

import nemo
import nemo.collections.asr as nemo_asr
from nemo.utils.exp_manager import exp_manager


In [58]:
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': ['ا', 'إ', 'أ', 'آ', 'ء', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ؤ', 'ي', 'ئ', 'ى', 'ً', 'ࣰ', 'ٌ', 'ࣱ', 'ٍ', 'ࣲ', 'َ', 'ُ', 'ِ', 'ّ', 'ٓ', 'ۤ', 'ٔ', 'ۥ', 'ۧ', 'ۡ', 'ۨ', 'ٰ', '۠', '۟', '۪', 'ٱ', '۬', '۫', 'ۢ', 'ۭ', 'ۜ', 'ۣ', ' '], 'model': {'train_ds': {'manifest_filepath': 'training_subset.json', 'sample_rate': 16000, 'labels': ['ا', 'إ', 'أ', 'آ', 'ء', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ؤ', 'ي', 'ئ', 'ى', 'ً', 'ࣰ', 'ٌ', 'ࣱ', 'ٍ', 'ࣲ', 'َ', 'ُ', 'ِ', 'ّ', 'ٓ', 'ۤ', 'ٔ', 'ۥ', 'ۧ', 'ۡ', 'ۨ', 'ٰ', '۠', '۟', '۪', 'ٱ', '۬', '۫', 'ۢ', 'ۭ', 'ۜ', 'ۣ', ' '], 'batch_size': 32, 'normalize_transcripts': False, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'num_workers': 0, 'pin_memory': True, '

In [59]:


#os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

checkpoint_callback = ModelCheckpoint(
    monitor='val_wer',  
    mode='min',     
    dirpath='checkpoints/', 
    filename='best_model-{epoch:02d}-{wer:.4f}',  
    save_top_k=1,   
    verbose=True,   
)

trainer = pl.Trainer(
    devices=1, 
    accelerator='gpu', 
    max_epochs=100, 
    callbacks=[checkpoint_callback],  
    logger = True
)

first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2024-07-06 23:29:04 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2024-07-06 23:29:04 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2024-07-06 23:29:12 collections:196] Dataset loaded with 59966 files totalling 148.63 hours
[NeMo I 2024-07-06 23:29:12 collections:197] 30034 files were filtered totalling 226.72 hours
[NeMo I 2024-07-06 23:29:12 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2024-07-06 23:29:12 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2024-07-06 23:29:13 collections:196] Dataset loaded with 7500 files totalling 32.66 hours
[NeMo I 2024-07-06 23:29:13 collections:197] 0 files were filtered t

In [60]:



checkpoint_path = './checkpoints/best_model-epoch=59-wer=0.0000.ckpt'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
first_asr_model.load_state_dict(checkpoint['state_dict'])
first_asr_model.eval()



# tensorboard --logdir=I:\Almaqraa\lightning_logs

EncDecCTCModel(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(64, 64, kernel_size=(11,), stride=(1,), padding=(5,), groups=64, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (3): SqueezeExcite(
            (fc): Sequential(
              (0): Linear(in_features=128, out_features=16, bias=False)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=16, out_features=128, bias=False)
            )
            (gap): AdaptiveAvgPool1d(output_size=1)
          )
        )
        (res): ModuleList(
          (0): ModuleList(
            (0): MaskedConv1d(
   

# WER calculations

In [61]:
params['model']['validation_ds']['batch_size'] = 8

first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
first_asr_model.cuda()
first_asr_model.eval()


wer_nums = []
wer_denoms = []


for test_batch in first_asr_model.test_dataloader():
    test_batch = [x.cuda() for x in test_batch]
    targets = test_batch[2]
    targets_lengths = test_batch[3]
    log_probs, encoded_len, greedy_predictions = first_asr_model(
        input_signal=test_batch[0], input_signal_length=test_batch[1]
    )

    first_asr_model._wer.update(
        predictions=greedy_predictions,
        targets=targets,
        target_lengths=targets_lengths,
    )

    _, wer_num, wer_denom = first_asr_model._wer.compute()
    first_asr_model._wer.reset()
    wer_nums.append(wer_num.detach().cpu().numpy())
    wer_denoms.append(wer_denom.detach().cpu().numpy())

    # Release tensors from GPU memory
    del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions
    torch.cuda.empty_cache()

print(f"WER = {sum(wer_nums) / sum(wer_denoms)}")

[NeMo I 2024-07-06 23:29:13 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2024-07-06 23:29:13 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2024-07-06 23:29:14 collections:196] Dataset loaded with 7500 files totalling 32.66 hours
[NeMo I 2024-07-06 23:29:14 collections:197] 0 files were filtered totalling 0.00 hours
WER = 0.5319547621687548


# Inference

In [62]:
data_dir = './validation_wav2/'



        
paths2audio_files = [os.path.join(data_dir, 'afasy.wav') ] #أُولَئِكَ عَلَى هُدًى مِنْ رَبِّهِمْ وَأُولَئِكَ هُمُ الْمُفْلِحُونَ 
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))                 

paths2audio_files = [os.path.join(data_dir, 'sample_23474.wav') ] ## الله الصمد
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))  

paths2audio_files = [os.path.join(data_dir, 'audio_25890.wav') ]# وَلَقَدْ آتَيْنَا دَاوُودَ وَسُلَيْمَانَ عِلْمًا ۖ وَقَالَا الْحَمْدُ لِلَّهِ الَّذِي فَضَّلَنَا عَلَىٰ كَثِيرٍ مِّنْ عِبَادِهِ الْمُؤْمِنِينَ
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1)) 

paths2audio_files = [os.path.join(data_dir, 'sample_7547.wav') ] #  فَبِأَيِّ آلَاءِ رَبِّكُمَا تُكَذِّبَانِ
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1)) 

paths2audio_files = [os.path.join(data_dir, 'long_verse.wav') ] 
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1)) 
#يَا أَيُّهَا الَّذِينَ آمَنُوا اذْكُرُوا نِعْمَةَ اللَّهِ عَلَيْكُمْ إِذْ جَاءَتْكُمْ جُنُودٌ فَأَرْسَلْنَا عَلَيْهِمْ رِيحًا وَجُنُودًا لَّمْ تَرَوْهَا ۚ وَكَانَ اللَّهُ بِمَا تَعْمَلُونَ بَصِيرًا


paths2audio_files = [os.path.join(data_dir, 'ayetEldayn.wav') ] 
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1)) 




Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['أُولَئِكَ عَلَى هُدًى مِن رَبِّهِم وَأُولَئِكَ هُمُ المُفلِحُونَ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['اللَّيُ الصَّمَدِ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

['فَبِأَيِّ آلَاءِ رَبِّكُمَا تُكَذِّبَانِ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['كَاأَيُّنَا الَّذِينَ آمَنُو الكُرُو نِعمَتَ اللَّهِ عَلَيكُم إِذ َّاأَدتُم جُنُولُن فَأَرفَلنَا عَلَيهِ لِيحَوا وَجُنُودًاَم كَرَوذًا وَكَالَ اللَّهُ بِمَا تَعمَنُونَ بَطِيرًا']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['يَا أَيُّهَا الَّذِينَ آمَنُوا إِذَا تَدَيَنتُم بِدَيهٍ إِلَى أَجَلٍ مُسَمَّ فَاككُرُوهُ وَليَكتُبَّينَكُم كَاتِ بُم بِالعَدل وَلَا يَأبَكَاتِ بٌ أَن يَك تُبَكَمَا عَلَّمَهُ اللَّهِ فَليَكتُّبوَليُملِلِ الَّذِي عَلَيهِ الحَقُّ وَليَتَّقِ الَّا هَرَبَّهُ وَلَا يَبخَس مِنهُ شَيئًا فَإِن كَانَ الَّذِي عَلَيهِ الحَقُّ سَفِينًا أَوضَعِيفًا أَولَا يَستَطِيعُ أَن يُمِ لَهُوَ فَ لُّم لِلوَلِيُّهُ بِالعَدِي وَاستَشِدُوا شَهِيدَينِ مِنرِّجَالِكُم فَإِن لَّم يَقُو نَارَجُلَينِ فَرَجُلُ وَامرَأَتَانِ مِنمَّن تَضَّونَ مِنَ الشُّ هَدًاءُ أَن تَضِلَّإِحدَاهُمَا فَُذَكِّرَ  إِحذَاهُمَا الأُخرَى وَلَا يَأبَششُّ هَدَاءُ إِذَا مَا دُوا وَلَا تَسأمُوا أَن تَكتُمُوهُ صَغِيرًا أَو كَبِيرًا إِلَى أَجَلِهِ ذَلِكُم أَقسَطُعِندَ اللَّهِ وَأَقوَمُلِالشَّهَادَِ وَأَدنَا أَلَّا تَرتَىبُوا إِلَّا أَن تَكُونَِجَارًَحَاضِرًَن تُدِيرُونَهَبَينَكُم فَلَيسَ عَلَيكُم جُنَا حُم أَلنَا تَكتُبُوهَا وَأَشهِدُوا إِذَا تَبَى يَعتُم وَلَا يُرَارَّكَاتِبُ وَلَا شَهِدِ وَإِن تَفعَلُوا فَإِنَّهُفُسُوقُمبِكُم وَاتَّقُاللَّهُ وَيُ

In [64]:
data_dir = './validation_wav2/'


paths2audio_files = [os.path.join(data_dir, 'sample_8279.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))


paths2audio_files = [os.path.join(data_dir, 'sample_8280.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))

paths2audio_files = [os.path.join(data_dir, 'sample_8312.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))


paths2audio_files = [os.path.join(data_dir, 'sample_9774.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,batch_size=1))



paths2audio_files = [os.path.join(data_dir, 'sample_9722.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))


paths2audio_files = [os.path.join(data_dir, 'sample_9494.wav') ]  
print(first_asr_model.transcribe(paths2audio_files=paths2audio_files,
                                 batch_size=1))

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['إِنَّ الَّذِينَ آمَنُوا وَعَمِلُوا الصَّالِحَاتِ لَهُم جَنَّاتُن تَ جرِي مِن تَحتِهَا ل أَنهَارِ ذَلِكَ الفَوزُو الكَ بِلِهِرِ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['بَلهُوَ قُرآنٌ مَجِيدٌ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

['وَأَقِيمُوا الصَّلَاَوَآتُوا الزَّكَاتَ وَأَطِيعُو الرَّسُونَ لَعَلَّكُم تُرحًمُونَ']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['والرَّحمَنُ عَلَى لعَرشِ استَوَى']


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['إِنَّا أَعطَينَاكَ الكَوثَر']
